# Simulating quantum programs on GPUs

In this notebook, you will learn how to run simulation jobs on GPUs with CUDA-Q using BYOC. You can learn about BYOC and view how to build a container image that supports CUDA-Q in the notebook "0_hello_cudaq_jobs.ipynb".

We start with necessary imports that are used in the examples below.

In [1]:
import os
import time
import numpy as np

from braket.jobs import hybrid_job
from braket.jobs.config import InstanceConfig
from braket.jobs.environment_variables import get_job_device_arn

from random_circuits import random_circuit_generator_factory

Specify the URI of the container image that supports CUDA-Q. If you went through the "0_hello_cudaq_jobs.ipynb" notebook, you can use the same image URI.

In [2]:
image_uri = "<image-uri>"

## Running jobs with GPUs
To use GPUs for circuit simulation, you can set the target backend to `nvidia`. You also need to select an instance that has NVIDIA GPUs. In the code snippet below, the instance type "ml.p3.2xlarge" is used as an example. The instance type "ml.p3.2xlarge" has a single NVIDIA V100 GPU. You can check [this page](https://aws.amazon.com/braket/pricing/) and [this page](https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs-configure-job-instance-for-script.html) to view available instances for Braket Hybrid Jobs. The `include_modules` keyword is set to `random_circuits` in order to use the random circuit generator defined in "random_circuits.py".

In [8]:
@hybrid_job(
    device='local:nvidia/nvidia',
    image_uri=image_uri,
    include_modules="random_circuits",
    instance_config=InstanceConfig(instanceType='ml.p3.2xlarge', instanceCount=1),
)
def gpu_job(n_qubits, n_gates, n_terms, n_shots):
    import cudaq

    # Define backend
    device = get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # Define circuit and observables
    get_random_circuit = random_circuit_generator_factory()
    circuit = get_random_circuit(n_qubits, n_gates)
    hamiltonian = cudaq.SpinOperator.random(n_qubits, n_terms)

    # Time the circuit simulation
    t0 = time.time()
    result = cudaq.observe(circuit, hamiltonian, shots_count=n_shots)
    t1 = time.time()
    print(f'result: {result.expectation()} | duration: {t1-t0}')

Skipping python version validation, make sure versions match between local environment and container.


When the `gpu_job` function is called, it creates a Braket job that runs on AWS, performing circuit simulations with the GPU.

In [ ]:
n_qubits = 20
n_gates = 150
n_terms = 2000
n_shots = 1000

job = gpu_job(n_qubits, n_gates, n_terms, n_shots)
print("Job ARN: ", job.arn)

## Tensor network backend
The `nvidia` backend shown in the example above is a statevector simulator. CUDA-Q also supports tensor network simulation with the `tensornet` backend. You can view [this page](https://nvidia.github.io/cuda-quantum/latest/using/backends/simulators.html#tensor-network-simulators) to learn more about the tensor network simulator. The list of CUDA-Q backends can be viewed in the [CUDA-Q documentation](https://nvidia.github.io/cuda-quantum/latest/using/backends/backends.html).

In [13]:
@hybrid_job(
    device='local:nvidia/tensornet',
    image_uri=image_uri,
    include_modules="random_circuits",
    instance_config=InstanceConfig(instanceType='ml.p3.2xlarge', instanceCount=1),
)
def gpu_job(n_qubits, n_gates, n_terms, n_shots):
    import cudaq

    # Define backend
    device = get_job_device_arn()
    cudaq.set_target(device.split('/')[-1])
    print("CUDA-Q backend: ", cudaq.get_target())

    # Define circuit and observables
    get_random_circuit = random_circuit_generator_factory()
    circuit = get_random_circuit(n_qubits, n_gates)
    hamiltonian = cudaq.SpinOperator.random(n_qubits, n_terms)

    # Time the circuit simulation
    t0 = time.time()
    result = cudaq.observe(circuit, hamiltonian, shots_count=n_shots)
    t1 = time.time()
    print(f'result: {result.expectation()} | duration: {t1-t0}')

Skipping python version validation, make sure versions match between local environment and container.


When circuits are shallow, the tensor network simulator can run circuits with high qubit count. For example, the code snippet below run a simulation of 50 qubits and the simulation can finish in a few seconds.

In [ ]:
n_qubits = 50
n_gates = 100
n_terms = 10
n_shots = 1000

job = gpu_job(n_qubits, n_gates, n_terms, n_shots)
print("Job ARN: ", job.arn)

## Summary
This notebook shows you how to target CUDA-Q GPU simulators for circuit simulation. In particular, this notebook shows an example of using a CUDA-Q statevector simulator,  the `nvidia` backend. If you have shallow circuits with high qubit count, you can use the CUDA-Q tensor network simulator, the `tensornet` backend, which may execute your circuits faster.